# merge、join、concat

* 参考
    * [pandasの使い方（merge、join、concat編）](http://pppurple.hatenablog.com/entry/2016/06/27/022310#concat関数)

# 事前準備

In [2]:
from pandas import DataFrame
#from pandas import Series, DataFrame
import pandas as pd

# merge

## 内部結合 (inner join)

### デフォルトでマージ

* 共通の同じ列名でマージされる
    * この場合はid列
    * 何も指定しなければinner joinになる
    * onでマージする列名を指定できる

In [121]:
df1 = DataFrame({'id': [1,2,3,4], 'owner_id': [3,1,1,4], 'dogs': ['Shiba', 'Akita', 'Poodle', 'Siberian Husky']})
df1.ix[:,['id','dogs','owner_id']]

,id,dogs,owner_id
0,1,Shiba,3
1,2,Akita,1
2,3,Poodle,1
3,4,Siberian Husky,4


In [122]:
df2 = DataFrame({'id': [1,2,3], 'owner': ['Taro', 'Jiro', 'Hanako']})
df2.ix[:,['id','owner']]

,id,owner
0,1,Taro
1,2,Jiro
2,3,Hanako


In [53]:
df_merge1 = pd.merge(df1, df2)
df_merge1

,dogs,id,owner_id,owner
0,Shiba,1,3,Taro
1,Akita,2,1,Jiro
2,Poodle,3,1,Hanako


In [54]:
### さっきと同じ
df_merge1 = pd.merge(df1, df2, how='inner')
df_merge1

,dogs,id,owner_id,owner
0,Shiba,1,3,Taro
1,Akita,2,1,Jiro
2,Poodle,3,1,Hanako


In [55]:
### これもさっきと同じ
df_merge1 = pd.merge(df1, df2, on='id')
df_merge1

,dogs,id,owner_id,owner
0,Shiba,1,3,Taro
1,Akita,2,1,Jiro
2,Poodle,3,1,Hanako


In [56]:
### ちゃんと全部書いた場合。結果はさっきと同じ
df_merge1 = pd.merge(df1, df2, how='inner', on='id')
df_merge1

,dogs,id,owner_id,owner
0,Shiba,1,3,Taro
1,Akita,2,1,Jiro
2,Poodle,3,1,Hanako


### それぞれのデータフレームの列を指定してマージ

* dogsのowner_idとownerのidでマージ
    * 列名が重複している場合
        * id_xはdogs側のid, id_yはowner側のid

In [57]:
df_merge2 = pd.merge(df1, df2, how='inner', left_on='owner_id', right_on='id')
df_merge2

,dogs,id_x,owner_id,id_y,owner
0,Shiba,1,3,3,Hanako
1,Akita,2,1,1,Taro
2,Poodle,3,1,1,Taro


In [82]:
### suffixをつけられる
df_merge2 = pd.merge(df1, df2, how='inner', left_on='owner_id', right_on='id', suffixes=('_left', '_right'))
df_merge2

,dogs,id_left,owner_id,id_right,owner
0,Shiba,1,3,3,Hanako
1,Akita,2,1,1,Taro
2,Poodle,3,1,1,Taro


In [58]:
### 必要な列のみ表示する
df_merge2.loc[:,['dogs','owner']]

,dogs,owner
0,Shiba,Hanako
1,Akita,Taro
2,Poodle,Taro


## 外部結合(outer join)

### 左外部結合

* howにleftを指定すると左外部結合になる
    * Siberian Huskeyは飼い主が居ないのでNaNになる

In [59]:
df_merge3 = pd.merge(df1, df2, how='left', left_on='owner_id', right_on='id')
df_merge3

,dogs,id_x,owner_id,id_y,owner
0,Shiba,1,3,3.0,Hanako
1,Akita,2,1,1.0,Taro
2,Poodle,3,1,1.0,Taro
3,Siberian Husky,4,4,NaN,NaN


### 右外部結合

* howにrightを指定すると右外部結合になる
    * Jiroは飼を飼っていないのでNaNになる

In [60]:
df_merge4 = pd.merge(df1, df2, how='right', left_on='owner_id', right_on='id')
df_merge4

,dogs,id_x,owner_id,id_y,owner
0,Shiba,1.0,3.0,3,Hanako
1,Akita,2.0,1.0,1,Taro
2,Poodle,3.0,1.0,1,Taro
3,NaN,NaN,NaN,2,Jiro


### 完全外部結合

* howにouterを指定すると右外部結合になる
    * Siberian Huskeyは飼い主が居ないのでNaNになる
    * Jiroは飼を飼っていないのでNaNになる

In [62]:
df_merge5 = pd.merge(df1, df2, how='outer', left_on='owner_id', right_on='id')
df_merge5

,dogs,id_x,owner_id,id_y,owner
0,Shiba,1.0,3.0,3.0,Hanako
1,Akita,2.0,1.0,1.0,Taro
2,Poodle,3.0,1.0,1.0,Taro
3,Siberian Husky,4.0,4.0,NaN,NaN
4,NaN,NaN,NaN,2.0,Jiro


## 複数キーでのマージ

In [66]:
df3 = DataFrame({'id': [1,2,3,4], 'owner_id': [3,1,1,4], 'dogs': ['Shiba', 'Akita', 'Poodle', 'Siberian Husky'], 'country': ['Japan', 'Japan', 'Germany', 'Siberian']})
df3.ix[:,['id','dogs','owner_id','country']]

,id,dogs,owner_id,country
0,1,Shiba,3,Japan
1,2,Akita,1,Japan
2,3,Poodle,1,Germany
3,4,Siberian Husky,4,Siberian


In [71]:
df4 = DataFrame({'id': [1,2,3], 'owner': ['Taro', 'Jiro', 'Hanako'], 'country': ['Japan', 'Japan', 'Japan']})
df4.ix[:,['id','owner','country']]

,id,owner,country
0,1,Taro,Japan
1,2,Jiro,Japan
2,3,Hanako,Japan


### 複数キーの内部結合

* countryとowner_id, idで結合
    * owner_idとidが一致し、かつcountryも一致している行が出力される

In [72]:
df_merge6 = pd.merge(df3, df4, how='inner', left_on=['owner_id','country'], right_on=['id','country'])
df_merge6

,country,dogs,id_x,owner_id,id_y,owner
0,Japan,Shiba,1,3,3,Hanako
1,Japan,Akita,2,1,1,Taro


### 複数キーの外部結合

In [73]:
df_merge7 = pd.merge(df3, df4, how='left', left_on=['owner_id','country'], right_on=['id','country'])
df_merge7

,country,dogs,id_x,owner_id,id_y,owner
0,Japan,Shiba,1,3,3.0,Hanako
1,Japan,Akita,2,1,1.0,Taro
2,Germany,Poodle,3,1,NaN,NaN
3,Siberian,Siberian Husky,4,4,NaN,NaN


In [74]:
df_merge8 = pd.merge(df3, df4, how='right', left_on=['owner_id','country'], right_on=['id','country'])
df_merge8

,country,dogs,id_x,owner_id,id_y,owner
0,Japan,Shiba,1.0,3.0,3,Hanako
1,Japan,Akita,2.0,1.0,1,Taro
2,Japan,NaN,NaN,NaN,2,Jiro


In [75]:
df_merge9 = pd.merge(df3, df4, how='outer', left_on=['owner_id','country'], right_on=['id','country'])
df_merge9

,country,dogs,id_x,owner_id,id_y,owner
0,Japan,Shiba,1.0,3.0,3.0,Hanako
1,Japan,Akita,2.0,1.0,1.0,Taro
2,Germany,Poodle,3.0,1.0,NaN,NaN
3,Siberian,Siberian Husky,4.0,4.0,NaN,NaN
4,Japan,NaN,NaN,NaN,2.0,Jiro


## indexとのマージ

In [111]:
df1_index = DataFrame(
    {'id': [1,2,3,4], 
     'owner_id': [3,1,1,4], 
     'dogs': ['Shiba', 'Akita', 'Poodle', 'Siberian Husky']
    })
df1_index = df1_index.set_index('id')
df1_index.ix[:,['dogs','owner_id']]

,dogs,owner_id
id,,
1,Shiba,3
2,Akita,1
3,Poodle,1
4,Siberian Husky,4


In [112]:
df2_index = DataFrame(
    {'id': [1,2,3], 
     'owner': ['Taro', 'Jiro', 'Hanako'],
     'num': ['1', '2', '3']
    })
df2_index = df2_index.set_index('id')
df2_index.ix[:,['owner','num']]

,owner,num
id,,
1,Taro,1
2,Jiro,2
3,Hanako,3


## indexと指定した列で結合

In [113]:
pd.merge(df1_index, df2_index, left_on='owner_id', right_index=True)

,dogs,owner_id,num,owner
id,,,,
1,Shiba,3,3,Hanako
2,Akita,1,1,Taro
3,Poodle,1,1,Taro


In [115]:
### 思ったように動いてない
pd.merge(df1_index, df2_index, right_on='num', left_index=True)

,dogs,owner_id,num,owner
id,,,,


## index同士で結合

In [117]:
pd.merge(df1_index, df2_index, how='inner', left_index=True, right_index=True)

,dogs,owner_id,num,owner
id,,,,
1,Shiba,3,1,Taro
2,Akita,1,2,Jiro
3,Poodle,1,3,Hanako


In [118]:
pd.merge(df1_index, df2_index, how='left', left_index=True, right_index=True)

,dogs,owner_id,num,owner
id,,,,
1,Shiba,3,1,Taro
2,Akita,1,2,Jiro
3,Poodle,1,3,Hanako
4,Siberian Husky,4,NaN,NaN


In [119]:
pd.merge(df1_index, df2_index, how='right', left_index=True, right_index=True)

,dogs,owner_id,num,owner
id,,,,
1,Shiba,3,1,Taro
2,Akita,1,2,Jiro
3,Poodle,1,3,Hanako


In [120]:
pd.merge(df1_index, df2_index, how='outer', left_index=True, right_index=True)

,dogs,owner_id,num,owner
id,,,,
1,Shiba,3,1,Taro
2,Akita,1,2,Jiro
3,Poodle,1,3,Hanako
4,Siberian Husky,4,NaN,NaN


# concat

In [132]:
s1 = DataFrame([1,2,3], columns=['s1'])
s1

,s1
0,1
1,2
2,3


In [133]:
s2 = DataFrame([4,5], columns=['s2'])
s2

,s2
0,4
1,5


In [134]:
s3 = DataFrame([6,7,8], columns=['s3'])
s3

,s3
0,6
1,7
2,8


In [135]:
pd.concat([s1, s2, s3])

,s1,s2,s3
0,1.0,NaN,NaN
1,2.0,NaN,NaN
2,3.0,NaN,NaN
0,NaN,4.0,NaN
1,NaN,5.0,NaN
0,NaN,NaN,6.0
1,NaN,NaN,7.0
2,NaN,NaN,8.0


In [139]:
s1 = DataFrame([1,2,3], columns=['s1'], index=['a','b','c'])
s2 = DataFrame([4,5], columns=['s2'], index=['d','e'])
s3 = DataFrame([6,7,8], columns=['s3'], index=['f','g','h'])
pd.concat([s1, s2, s3])

,s1,s2,s3
a,1.0,NaN,NaN
b,2.0,NaN,NaN
c,3.0,NaN,NaN
d,NaN,4.0,NaN
e,NaN,5.0,NaN
f,NaN,NaN,6.0
g,NaN,NaN,7.0
h,NaN,NaN,8.0


In [140]:
s1 = DataFrame([1,2,3], columns=['data'], index=['a','b','c'])
s2 = DataFrame([4,5], columns=['data'], index=['d','e'])
s3 = DataFrame([6,7,8], columns=['s3'], index=['f','g','h'])
pd.concat([s1, s2, s3])

,data,s3
a,1.0,NaN
b,2.0,NaN
c,3.0,NaN
d,4.0,NaN
e,5.0,NaN
f,NaN,6.0
g,NaN,7.0
h,NaN,8.0


In [141]:
s1 = DataFrame([1,2,3], columns=['data'], index=['a','b','c'])
s2 = DataFrame([4,5], columns=['data'], index=['d','e'])
s3 = DataFrame([6,7,8], columns=['data'], index=['f','g','h'])
pd.concat([s1, s2, s3])

,data
a,1
b,2
c,3
d,4
e,5
f,6
g,7
h,8


### 行で結合

* axis=0を指定
    * デフォルトと同じ

In [142]:
s1 = DataFrame([1,2,3], columns=['data'], index=['a','b','c'])
s2 = DataFrame([4,5], columns=['data'], index=['d','e'])
s3 = DataFrame([6,7,8], columns=['data'], index=['f','g','h'])
pd.concat([s1, s2, s3], axis=0)

,data
a,1
b,2
c,3
d,4
e,5
f,6
g,7
h,8


### 列で結合

* axis=1を指定

In [147]:
s1 = DataFrame([1,2,3], columns=['data'], index=['a','b','c'])
s2 = DataFrame([4,5], columns=['data'], index=['d','e'])
s3 = DataFrame([6,7,8], columns=['data'], index=['f','g','h'])
pd.concat([s1, s2, s3], axis=1)

,data,data,data
a,1.0,NaN,NaN
b,2.0,NaN,NaN
c,3.0,NaN,NaN
d,NaN,4.0,NaN
e,NaN,5.0,NaN
f,NaN,NaN,6.0
g,NaN,NaN,7.0
h,NaN,NaN,8.0


In [152]:
s1 = DataFrame([1,2,3], columns=['s1'], index=['h','b','c'])
s2 = DataFrame([4,5], columns=['s2'], index=['b','a'])
s3 = DataFrame([6,7,8], columns=['s3'], index=['b','c','h'])
pd.concat([s1, s2, s3], axis=1)

,s1,s2,s3
a,NaN,5.0,NaN
b,2.0,4.0,6.0
c,3.0,NaN,7.0
h,1.0,NaN,8.0


In [167]:
s1 = DataFrame([1,2,3], columns=['s1'], index=['a','b','c'])
s2 = DataFrame([4,5,9], columns=['s2'], index=['a','b','c'])
s3 = DataFrame([6,7,8], columns=['s3'], index=['a','b','c'])
pd.concat([s1, s2, s3], axis=1)

,s1,s2,s3
a,1,4,6
b,2,5,7
c,3,9,8
